# Chatbot using Neural Networks and Natural Language Processing


## Abstract:
In this project I will be making a retrieval based Chatbot using NLTK and Keras. I will be also coding a Graphical User Interface which will predict responses from the bot using the Tkinter library. The chatbot will be trained on a dataset containing categories, patterns and responses. I will create a neural network to classify which category the user's message belongs to and give random response from the list of responses.

# Importing and loading the data file

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

Using TensorFlow backend.


# Necessary packages to run the code

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Pre processing the data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


# Creating training and testing data

In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# Building the model

In [5]:
# Model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
47/47 [==============================] - 2s 46ms/step - loss: 2.2469 - accuracy: 0.0851
Epoch 2/200
47/47 [==============================] - 0s 639us/step - loss: 2.1194 - accuracy: 0.1915
Epoch 3/200
47/47 [==============================] - 0s 642us/step - loss: 1.9999 - accuracy: 0.2553
Epoch 4/200
47/47 [==============================] - 0s 851us/step - loss: 1.9845 - accuracy: 0.3617
Epoch 5/200
47/47 [==============================] - 0s 630us/step - loss: 1.7863 - accuracy: 0.5319
Epoch 6/200
47/47 [==============================] - 0s 853us/step - loss: 1.7367 - accuracy: 0.5319
Epoch 7/200
47/47 [==============================] - 0s 850us/step - loss: 1.5823 - accuracy: 0.5106
Epoch 8/200
47/47 [==============================] - 0s 858us/step - loss: 1.3029 - accuracy: 0.6809
Epoch 9/200
47/47 [==============================] - 0s 637us/step - loss: 1.2417 - accuracy: 0.6596
Epoch 10/200
47/47 [==============================] - 0s 635us/step - loss: 1.2009 - accurac

47/47 [==============================] - 0s 643us/step - loss: 0.0194 - accuracy: 1.0000
Epoch 81/200
47/47 [==============================] - 0s 514us/step - loss: 0.0516 - accuracy: 0.9787
Epoch 82/200
47/47 [==============================] - 0s 544us/step - loss: 0.1303 - accuracy: 0.9362
Epoch 83/200
47/47 [==============================] - 0s 647us/step - loss: 0.0467 - accuracy: 1.0000
Epoch 84/200
47/47 [==============================] - 0s 628us/step - loss: 0.0470 - accuracy: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 481us/step - loss: 0.0397 - accuracy: 1.0000
Epoch 86/200
47/47 [==============================] - 0s 635us/step - loss: 0.0558 - accuracy: 0.9787
Epoch 87/200
47/47 [==============================] - 0s 546us/step - loss: 0.0423 - accuracy: 0.9787
Epoch 88/200
47/47 [==============================] - 0s 514us/step - loss: 0.0463 - accuracy: 0.9787
Epoch 89/200
47/47 [==============================] - 0s 432us/step - loss: 0.0149 - accuracy: 

47/47 [==============================] - 0s 1ms/step - loss: 0.0299 - accuracy: 0.9787
Epoch 160/200
47/47 [==============================] - 0s 847us/step - loss: 0.0207 - accuracy: 1.0000
Epoch 161/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0929 - accuracy: 0.9574
Epoch 162/200
47/47 [==============================] - 0s 849us/step - loss: 0.0460 - accuracy: 0.9787
Epoch 163/200
47/47 [==============================] - 0s 843us/step - loss: 0.1299 - accuracy: 0.9574
Epoch 164/200
47/47 [==============================] - 0s 851us/step - loss: 0.0322 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 959us/step - loss: 0.0211 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 2ms/step - loss: 0.0785 - accuracy: 0.9787
Epoch 167/200
47/47 [==============================] - 0s 847us/step - loss: 0.0214 - accuracy: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 718us/step - loss: 0.0412 - accurac

# Predicting the response

We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.

In [6]:

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class.



In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

After predicting the class, we will get a random response from the list of intents.

In [8]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

# Creating Graphical User Interface with Tkinter

In [10]:

import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Chatbot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()